In [11]:
library(tidyverse)
library(tidymodels)
library(repr)
library(cowplot)
options(repr.matrix.max.rows = 6)
source('cleanup.R')

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'cleanup.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [12]:
players <- read_csv("https://raw.githubusercontent.com/ansonansonnn/project-data/refs/heads/main/players.csv")
players

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,57
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


In [13]:
players_short <- players |>
    select(experience, Age, subscribe) |>
    drop_na()
players_short

experience,Age,subscribe
<chr>,<dbl>,<lgl>
Pro,9,TRUE
Veteran,17,TRUE
Veteran,17,FALSE
⋮,⋮,⋮
Veteran,22,FALSE
Amateur,57,FALSE
Amateur,17,FALSE


In [14]:
set.seed(1000)

# Optimal K

players_short <- players_short |>
    mutate(subscribe = as.factor(subscribe))

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

data_split <- initial_split(players_short, prop = 0.7, strata = subscribe) 
    train_data <- training(data_split)
    test_data  <- testing(data_split)

knn_recipe <- recipe(subscribe ~ Age, data = train_data) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

knn_wf <- workflow() |>
    add_model(knn_spec) |>
    add_recipe(knn_recipe)

k_grid <- grid_regular(neighbors(range = c(1, 10)), levels = 10)

k_vfold <- vfold_cv(train_data, v = 5)

knn_tuned <- tune_grid(knn_wf, resamples = k_vfold, grid = k_grid, metrics = metric_set(accuracy))

k_select <- select_best(knn_tuned, metric = "accuracy")
k_select

neighbors,.config
<int>,<chr>
7,Preprocessor1_Model07


In [15]:
sub_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = k_select) |>
    set_engine("kknn") |>
    set_mode("classification")

sub_recipe <- recipe(subscribe ~ Age, data = players_short) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

sub_workflow <- workflow() |>
    add_model(sub_spec) |>
    add_recipe(sub_recipe)

sub_workflow

══ Workflow ════════════════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: nearest_neighbor()

── Preprocessor ────────────────────────────────────────────────────────────────
2 Recipe Steps

• step_scale()
• step_center()

── Model ───────────────────────────────────────────────────────────────────────
K-Nearest Neighbor Model Specification (classification)

Main Arguments:
  neighbors = k_select
  weight_func = rectangular

Computational engine: kknn 


In [ ]:
set.seed(123)

players_class <- players_clean |>
mutate(subscribe= factor(subscribe, levels= c(FALSE, TRUE)),
       experience_num= as.numeric((factor(experience, 
                                       levels = c("Beginner", "Amateur", "Regular", "Pro", "Veteran"))))
) |> select(subscribe, Age, experience_num)
    

players_split <- initial_split(players_class, prop= 0.8)
players_train <- training(players_split)
players_test <- testing(players_split)


